# Day 4, part 2: Outbreak estimates

## Introduction

This practical is the second (out of two) part of a practical which simulates the early assessment and reconstruction of an Ebola Virus Disease (EVD) outbreak. 
Please make sure you have gone through **part 1** before starting **part 2**. 
In **part 2** of the practical, we introduce various aspects of analysis of the early stage of an outbreak, including growth rate estimation, contact tracing data, delays, and estimates of transmissibility. 


## Learning outcomes

By the end of this practical **part 2**, you should be able to:

- Estimate & interpret the growth rate & doubling time of the epidemic

- Estimate the serial interval from data on pairs infector / infected individuals

- Estimate & interpret the reproduction number of the epidemic

- Forecast short-term future incidence

## Context: A novel EVD outbreak in a fictional country in West Africa

A new EVD outbreak has been notified in a fictional country in West Africa. The Ministry of Health is in charge of coordinating the outbreak response, and have contracted you as a consultant in epidemic analysis to inform the response in real time. You have already read in an done descriptive analysis of the data (**part 1** of the practical). Now let's do some statistical analyses! 

## Required packages 

The following packages, available on CRAN or github, are needed for this analysis.
You should have installed them in **part 1** but if not, install necessary packages as follows:

Once the packages are installed, you may need to open a new R session. Then load the libraries as follows:

In [ ]:
library(readxl)
library(outbreaks)
library(incidence)
library(epicontacts)
library(distcrete)
library(epitrix)
library(EpiEstim)
library(projections)
library(ggplot2)
library(magrittr)
library(binom)
library(ape)
library(outbreaker2)
library(here)

## Read in the data processed in [part 1](./real-time-response-1.html)

<!--
ZNK: These two chunks are needed because of the way the documents are structured
in blogdown. The source that we edit is not the same as the site that is
rendered. Everything in this directory will end up in the same directory as the
"static" when the website is displayed, but the current directory structure is
present when the practicals are built, so we have to do this silly 
song-and-dance to appease the directory gods.
-->

In [ ]:
i_daily <- readRDS(here("data/clean/i_daily.rds"))
i_weekly <- readRDS(here("data/clean/i_weekly.rds"))
linelist <- readRDS(here("data/clean/linelist.rds"))
linelist_clean <- readRDS(here("data/clean/linelist_clean.rds"))
contacts <- readRDS(here("data/clean/contacts.rds"))

## Estimating the growth rate using a log-linear model

The simplest model of incidence is probably the log-linear model, i.e. a linear 
regression on log-transformed incidences. 
For this we will work with weekly incidence, to avoid having
too many issues with zero incidence (which cannot be logged). 

Visualise the log-transformed incidence:

In [ ]:
ggplot(as.data.frame(i_weekly)) + 
  geom_point(aes(x = dates, y = log(counts))) + 
  scale_x_incidence(i_weekly) +
  xlab("date") +
  ylab("log weekly incidence") + 
  theme_minimal()

What does this plot tell you about the epidemic?   

In the `incidence` package, the function `fit` will estimate the parameters of this model from an incidence object (here, `i_weekly`). Apply it to the data and store the result in a new object called `f`. You can print and use `f` to derive estimates of the growth rate r and the doubling time, and add the corresponding model to the incidence plot:  

Fit a log-linear model to the weekly incidence data:  

In [ ]:
f <- incidence::fit(i_weekly)
f
plot(i_weekly, fit = f)

Looking at the plot and fit, do you think this is a reasonable fit?   

## Finding a suitable threshold date for the log-linear model, based on the observed delays

Using the plot of the log(incidence) that you plotted above, and thinking about why exponential growth may not be observed in the most recent weeks, choose a threshold date and fit the log-linear model to a suitable section of the epicurve where you think we can more reliably estimate the growth rate, r, and the doubling time.  

You may want to examine how long after onset of symptoms cases are hospitalised; 
this may inform the threshold date you choose, as follows:  

In [ ]:
summary(as.numeric(linelist_clean$date_of_hospitalisation - linelist_clean$date_of_onset))

In [ ]:
# how many weeks should we discard at the end of the epicurve
n_weeks_to_discard <- 

In [ ]:
min_date <- min(i_daily$dates)
max_date <- max(i_daily$dates) - n_weeks_to_discard * 7
# weekly truncated incidence
i_weekly_trunc <- subset(i_weekly, 
                         from = min_date, 
                         to = max_date) # discard last few weeks of data
# daily truncated incidence (not used for the linear regression but may be used later)
i_daily_trunc <- subset(i_daily, 
                         from = min_date, 
                         to = max_date) # remove last two weeks of data

Refit and plot your log-linear model as before but using the truncated data `i_weekly_trunc`. 

In [ ]:
f <- incidence::fit(i_weekly_trunc)
f
plot(i_weekly_trunc, fit = f)

Look at the summary statistics of your fit:

In [ ]:
summary(f$model)

You can look at the goodness of fit (Rsquared), the estimated slope (growth rate)
and the corresponding doubling time as below:  

In [ ]:
# does the model fit the data well? 
adjRsq_model_fit <- summary(f$model)$adj.r.squared
# what is the estimated growth rate of the epidemic?
daily_growth_rate <- f$model$coefficients['dates.x']
# confidence interval:
daily_growth_rate_CI <- confint(f$model, 'dates.x', level=0.95)
# what is the doubling time of the epidemic? 
doubling_time_days <- log(2) / daily_growth_rate
# confidence interval:
doubling_time_days_CI <- log(2) / rev(daily_growth_rate_CI)


Although the log-linear is a simple and quick method for early epidemic assessment, care must be taken to only fit to the point that there is epidemic growth. Note that it may be difficult to define this point.  

## Contact Tracing - Looking at contacts

Contact tracing is one of the pillars of an Ebola outbreak response. This involves identifying and following up any at risk individuals who have had contact with a known case (i.e. may have been infected). For Ebola, contacts are followed up for 21 days (the upper bound of the incubation period). This ensures that contacts who become symptomatic can be isolated quickly, reducing the chance of further transmission. We use the full linelist here rather than `linelist_clean` where we discarded entries with errors in the dates. However, the contact may still be valid.

Using the function `make_epicontacts` in the `epicontacts` package, create a new `epicontacts` object called `epi_contacts`. Make sure you check the column names of the relevant `to` and `from` arguments. 

In [ ]:
epi_contacts <- make_epicontacts(linelist = linelist, contacts = contacts, 
                                 id = "case_id", # name of identifier in linelist
                                 from = "infector", # name of 'from' col in the contacts
                                 to = "case_id",  # name of 'to' col in the contacts
                                 directed = TRUE)

In [ ]:
epi_contacts

You can easily plot these contacts, but with a little bit of tweaking (see `?vis_epicontacts`) you can customise for example shapes by gender and arrow colours by source of exposure (or other variables of interest):

In [ ]:
# for example, look at the reported source of infection of the contacts.
table(epi_contacts$contacts$source, useNA = "ifany")


In [ ]:
p <- plot(epi_contacts, node_shape = "gender", shapes = c(m = "male", f = "female"), node_color = "gender", edge_color = "source", selector = FALSE)
p

In [ ]:
p <- plot(epi_contacts, node_shape = "gender", shapes = c(m = "male", f = "female"), node_color = "gender", edge_color = "source", selector = FALSE)
#learn::save_and_use_widget(p, 'real-time-response-2-nework.html')

Using the function `match` (see `?match`) check that the visualised contacts are indeed cases.

In [ ]:
match(contacts$case_id, linelist$case_id)

Once you are happy that these are all indeed cases, look at the network:  

- does it look like there is super-spreading (heterogeneous transmission)?
- looking at the gender of the cases, can you deduce anything from this? Are there any visible differences by gender?

## Estimating transmissibility (`$R$`)  

### Branching process model  

The transmissibility of the disease can be assessed through the estimation of the reproduction number R, defined as the expected number of secondary cases per infected case. In the early stages of an outbreak, and assuming a large population with no immunity, this quantity is also the basic reproduction number $$R_0$$, i.e. $$R$$ in a large fully susceptible population.  

The package `EpiEstim` implements a Bayesian estimation of $$R$$, using dates of onset of symptoms and information on the serial interval distribution, i.e. the distribution of time from symptom onset in a case and symptom onset in their infector (see Cori et al., 2013, AJE 178: 1505–1512).

Briefly, `EpiEstim` uses a simple model describing incidence on a given day as Poisson distributed, with a mean determined by the total force of infection on that day:

$$ I_t  ∼  Poisson(\lambda_t)$$

where $$I_t$$ is the incidence (based on symptom onset) on day $$t$$ and $$\lambda_t$$ is the force of infection on that day. Noting R the reproduction number and w() the discrete serial interval distribution, we have:

 $$\lambda_t = R \sum_{s=1}^t I_sw(t-s)$$

The likelihood (probability of observing the data given the model and parameters) is defined as a function of R:

 $$L(I) = p(I|R) = \prod_{t = 1}^{T} f_P(I_t, \lambda_t)$$

where $$f_P(.,\mu)$$ is the probability mass function of a Poisson distribution with mean $$\mu$$.

### Estimating the serial interval

As data was collected on pairs of infector and infected individuals, this should be sufficient to estimate the serial interval distribution. If that was not the case, we could have used data from past outbreaks instead.  

Use the function `get_pairwise` can be used to extract the serial interval (i.e. the difference in date of onset between infectors and infected individuals):

In [ ]:
si_obs <- get_pairwise(epi_contacts, "date_of_onset")
summary(si_obs)
## Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
## 1.000   5.000   6.500   9.117  12.250  30.000 
hist(si_obs, breaks = 0:30,
     xlab = "Days after symptom onset", ylab = "Frequency",
     main = "Serial interval (empirical distribution)",
     col = "grey", border = "white")


What do you think of this distribution? Make any adjustment you would deem necessary, and then use the function `fit_disc_gamma` from the package `epitrix` to fit a discretised Gamma distribution to these data. Your results should approximately look like:  

In [ ]:
si_fit <- fit_disc_gamma(si_obs, w = 1)
si_fit

`si_fit` contains various information about the fitted delays, including the estimated distribution in the `$distribution` slot. You can compare this distribution to the empirical data in the following plot:

In [ ]:
si <- si_fit$distribution
si

## compare fitted distribution
hist(si_obs, xlab = "Days after symptom onset", ylab = "Frequency",
     main = "Serial interval: fit to data", col = "salmon", border = "white",
     50, ylim = c(0, 0.15), freq = FALSE, breaks = 0:35)
points(0:60, si$d(0:60), col = "#9933ff", pch = 20)
points(0:60, si$d(0:60), col = "#9933ff", type = "l", lty = 2)


Would you trust this estimation of the generation time? How would you compare it to actual estimates from the West African EVD outbreak (WHO Ebola Response Team (2014) NEJM 371:1481–1495) with a mean of 15.3 days and a standard deviation 9.3 days?  

### Estimation of the Reproduction Number

Now that we have estimates of the serial interval, we can use this information to estimate transmissibility of the disease (as measured by `$R_0$`). Make sure you use a daily (not weekly) incidence object truncated to the period where you have decided there is exponential growth (`i_daily_trunc`).  

Using the estimates of the mean and standard deviation of the serial interval you just obtained, use the function `estimate_R` to estimate the reproduction number  (see `?estimate_R`) and store the result in a new object `R`.  

Before using `estimate_R`, you need to create a `config` object using the `make_config` function, where you will specify the time window where you want to estimate the reproduction number as well as the `mean_si` and `std_si` to use. For the time window, use `t_start = 2` (you can only estimate R from day 2 onwards as you are conditioning on the past observed incidence) and specify `t_end = length(i_daily_trunc$counts)` to estimate R up to the last date of your truncated incidence `i_daily_trunc`.  

In [ ]:

config <- make_config(mean_si = si_fit$mu, # mean of the si distribution estimated earlier
                      std_si = si_fit$sd,  # standard deviation of si dist estimated earlier
                      t_start = 2,         # starting day of time window
                      t_end = length(i_daily_trunc$counts)) # final day of time window


In [ ]:
R <- estimate_R(i_daily_trunc, method = "parametric_si", config = config)
plot(R, legend = FALSE)

Extract the median and 95% credible intervals (95%CrI) for the reproduction number as follows:

In [ ]:
R_median <- R$R$`Median(R)`
R_CrI <- c(R$R$`Quantile.0.025(R)`, R$R$`Quantile.0.975(R)`)
R_median
R_CrI

Interpret these results: what do you make of the reproduction number? What does it reflect? Based on the last part of the epicurve, some colleagues suggest that incidence is going down and the outbreak may be under control. What is your opinion on this?

Note that you could have estimated R0 directly from the growth rate and the serial interval, 
using the formula described in Wallinga and Lipsitch, Proc Biol Sci, 2007: 
$$R_0 = \frac{1}{\int_{s=0}^{+\infty}e^{-rs}w(s)ds}$$ , and implemented in the function `r2R0` of the `epitrix` package. Although this may seem like a complicated formula, the reasoning behind it is simple and illustrated in the figure below: for an observed exponentially growing incidence curve, 
if you know the serial interval, you can derive the reproduction number. 

![Estimating R0 from the growth rate and the serial interval.](../static/img/R0fromr.png)

Compared to the figure above, there is uncertainty in the growth rate r, and the serial interval has a full distribution rather than a single value. This can be accounted for in estimating R as follows:

In [ ]:
# generate a sample of R0 estimates from the growth rate and serial interval we estimated above 
R_sample_from_growth_rate <- lm2R0_sample(f$model, # log-linear model which contains our estimates of the growth rate r
                                          si$d(1:100), # serial interval distribution (truncated after 100 days)
                                          n = 1000) # desired sample size
# plot these:
hist(R_sample_from_growth_rate)
# what is the median?
R_median_from_growth_rate <- median(R_sample_from_growth_rate)
R_median_from_growth_rate # compare with R_median
# what is the 95%CI?
R_CI_from_growth_rate <- quantile(R_sample_from_growth_rate, c(0.025, 0.975))
R_CI_from_growth_rate # compare with R_CrI

Note the above estimates are slighlty different from those obtained using the branching process model. 
There are a few reasons for this. 
First, you used more detailed data (daily vs weekly incidence) for the branching process (EpiEstim) estimate. 
Furthermore, the log-linear model puts the same weight on all data points, whereas the branching process model puts a different weight on each data point (depending on the incidence observed at each time step). 
This may lead to slighlty different R estimates. 

## Short-term incidence forecasting  

The function `project` from the package `projections` can be used to simulate plausible epidemic trajectories by simulating daily incidence using the same branching process as the one used to estimate `$R$` in `EpiEstim`. All that is needed is one, or several values of `$R$` and a serial interval distribution, stored as a `distcrete` object.  

Here, we first illustrate how we can simulate 5 random trajectories using a fixed value of `$R$` = `r signif(R_median, 3)`, the median estimate of R from above.  
Use the same daily truncated incidence object as above to simulate future incidence.  

In [ ]:
#?project
small_proj <- project(i_daily_trunc,# incidence object 
                      R = R_median, # R estimate to use
                      si = si,      # serial interval distribution
                      n_sim = 5,    # simulate 5 trajectories
                      n_days = 10,  # over 10 days
                      R_fix_within = TRUE) # keep the same value of R every day

# look at each projected trajectory (as columns):
as.matrix(small_proj)

Using the same principle, generate 1,000 trajectories for the next 2 weeks, using a range of plausible values of `$R$`.  
The posterior distribution of R is gamma distributed (see Cori et al. AJE 2013) so you can use the `rgamma` function to randomly draw values from that distribution. You will also need to use the function `gamma_mucv2shapescale` from the `epitrix` package as shown below.  

In [ ]:

sample_R <- function(R, n_sim = 1000)
{
  mu <- R$R$`Mean(R)`
  sigma <- R$R$`Std(R)`
  Rshapescale <- gamma_mucv2shapescale(mu = mu, cv = sigma / mu)
  R_sample <- rgamma(n_sim, shape = Rshapescale$shape, scale = Rshapescale$scale)
  return(R_sample)
}

R_sample <- sample_R(R, 1000) # sample 1000 values of R from the posterior distribution
hist(R_sample, col = "grey")  # plot histogram of sample
abline(v = R_median, col = "red") # show the median estimated R as red solid vertical line
abline(v = R_CrI, col = "red", lty = 2) # show the 95%CrI of R as red dashed vertical lines

Store the results of your new projections in an object called `proj`.

In [ ]:
proj <- project(x = i_daily_trunc, 
                R = R_sample, # now using a sample of R values
                si = si, 
                n_sim = 1000, 
                n_days = 14, # project over 2 weeks
                R_fix_within = TRUE)


You can visualise the projections as follows:  

In [ ]:
plot(i_daily_trunc) %>% add_projections(proj, c(0.025, 0.5, 0.975))

How would you interpret the following summary of the projections?  

In [ ]:
apply(proj, 1, summary)
apply(proj, 1, function(x) mean(x > 0)) # proportion of trajectories with at least 
                                        # one case on each given day

apply(proj, 1, mean) # mean daily number of cases 

apply(apply(proj, 2, cumsum), 1, summary) # projected cumulative number of cases in 
                                          # the next two weeks

According to these results, what are the chances that more cases will appear in the near future? Is this outbreak being brought under control? Would you recommend scaling up / down the response? Is this consistent with your estimate of $$R$$?
 

## Pause !

Please let a demonstrator know when you have reached this point before proceeding further. 

## Accounting for uncertainty in the serial interval estimates when estimating the reproduction number

Note that this section is independent from the following ones, please skip if you don'thave time. 

EpiEstim allows to explicitly account for the uncertainty in the serial interval estimates because of limited sample size of pairs of infector/infected individuals. Note that it also allows accounting for uncertainty on the dates of symptom onset for each of these pairs (which is not needed here). 

Use the `method = "si_from_data"` option in `estimate_R`. 
To use this option, you need to create a data frame with 4 columns: 
`EL`, `ER`, `SL` and `SR` for the left (L) and right (R) bounds of the observed time of symptoms in the infector (E) and infected (S for secondary) cases. Here we derive this from `si_obs` as follows:

In [ ]:
si_data <- data.frame(EL = rep(0L, length(si_obs)), 
                      ER = rep(1L, length(si_obs)), 
                      SL = si_obs, SR = si_obs + 1L)

We can then feed this into `estimate_R` (but this will take some time to run as it estimates the serial interval distribution using an MCMC and fully accounts for the uncertainty in the serial interval to estimate the reproduction number).

In [ ]:
config <- make_config(t_start = 2, 
                      t_end = length(i_daily_trunc$counts))
R_variableSI <- estimate_R(i_daily_trunc, method = "si_from_data", 
                           si_data = si_data,
                           config = config)
# checking that the MCMC converged: 
R_variableSI$MCMC_converged
# and plotting the output:
plot(R_variableSI)

Look at the new median estimated R and 95%CrI: how different are they from your previous estimates? Do you think the size of the contacts dataset has had an impact on your results? 

In [ ]:
R_variableSI_median <- R_variableSI$R$`Median(R)`
R_variableSI_CrI <- c(R_variableSI$R$`Quantile.0.025(R)`, R_variableSI$R$`Quantile.0.975(R)`)
R_variableSI_median
R_variableSI_CrI

## Estimating time-varying transmissibility  

When the assumption that ($$R$$) is constant over time becomes untenable, an alternative is to estimating time-varying transmissibility using the instantaneous reproduction number ($$R_t$$). This approach, introduced by Cori et al. (2013), is also implemented in the package `EpiEstim`. It estimates ($$R_t$$) for a custom time windows (default is a succession of sliding time windows), using the same Poisson likelihood described above. In the following, we estimate transmissibility for 1-week sliding time windows (the default of `estimate_R`):  

In [ ]:
config = make_config(list(mean_si = si_fit$mu, std_si = si_fit$sd))  
# t_start and t_end are automatically set to estimate R over 1-week sliding windows by default. 

In [ ]:
Rt <- estimate_R(incid = i_daily_trunc,      # incidence object
                 method = "parametric_si",   # use parametric serial interval
                 config = config)            # config specified above

# look at the most recent Rt estimates:
tail(Rt$R[, c("t_start", "t_end", "Median(R)", 
             "Quantile.0.025(R)", "Quantile.0.975(R)")])

Plot the estimate of `$R$` over time:

In [ ]:
plot(Rt, legend = FALSE)

How would you interpret this result? What is the caveat of this representation?

What would you have concluded if instead of using `i_daily_trunc` as above, you had used the entire epidemics curve io.e. `i_daily`? 

In [ ]:
Rt_whole_incid <- estimate_R(incid = i_daily, 
                             method = "parametric_si", 
                             config = config) 

tail(Rt_whole_incid$R[, c("t_start", "t_end", 
                         "Median(R)", "Quantile.0.025(R)", "Quantile.0.975(R)")])  
# the above incorrectly infers that the recent transmissibility is <1

In [ ]:
cat("- the above assumes constant R within sliding time-window")  
